In [29]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from obspy import read
import os
from pprint import pprint
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import matplotlib.pyplot as plt


In [5]:
test_data_path = "./data/lunar/test/data/S12_GradeB"
train_data_path = "./data/lunar/training/data/S12_GradeA"

In [8]:
test_files = os.listdir(test_data_path)
count_test_files = len(test_files)
train_files = os.listdir(train_data_path)
count_train_files = len(train_files)


In [16]:
pprint(test_files[:5])
pprint(train_files[:5])

['.ipynb_checkpoints',
 'xa.s12.00.mhz.1969-12-16HR00_evid00006.csv',
 'xa.s12.00.mhz.1969-12-16HR00_evid00006.mseed',
 'xa.s12.00.mhz.1970-01-09HR00_evid00007.csv',
 'xa.s12.00.mhz.1970-01-09HR00_evid00007.mseed']
['.ipynb_checkpoints',
 'xa.s12.00.mhz.1970-01-19HR00_evid00002.csv',
 'xa.s12.00.mhz.1970-01-19HR00_evid00002.mseed',
 'xa.s12.00.mhz.1970-03-25HR00_evid00003.csv',
 'xa.s12.00.mhz.1970-03-25HR00_evid00003.mseed']


In [ ]:
train_csv = pd.read_csv(f'{test_data_path}/{test_files[1]}')
test_csv = pd.read_csv(f'{train_data_path}/{train_files[1]}')

# Загрузка mseed
train_mseed = read(f'{test_data_path}/{test_files[2]}')
test_mseed = read(f'{train_data_path}/{train_files[2]}')

# Пример того, как получить данные из mseed
train_stream = train_mseed[0].data
test_stream = test_mseed[0].data

In [ ]:
train_csv.head()

In [ ]:
test_csv.head()

In [ ]:
train_csv['time_abs'] = pd.to_datetime(train_csv['time_abs(%Y-%m-%dT%H:%M:%S.%f)'])

scaler = StandardScaler()
train_csv['velocity_normalized'] = scaler.fit_transform(train_csv[['velocity(m/s)']])
test_csv['velocity_normalized'] = scaler.fit_transform(test_csv[['velocity(m/s)']])


In [ ]:
train_csv.head()

In [ ]:
def create_dataset(data, window_size=100):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i + window_size])
        y.append(data[i + window_size]) 
    return np.array(X), np.array(y)

window_size = 100
X_train, y_train = create_dataset(train_csv['velocity_normalized'].values, window_size)
X_test, y_test = create_dataset(test_csv['velocity_normalized'].values, window_size)

In [ ]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(window_size, 1)))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))



In [ ]:
y_pred = model.predict(X_test)

plt.plot(y_test, label='Original')
plt.plot(y_pred, label='Denoised')
plt.legend()
plt.show()